# 🧹 02_Limpieza de Datos



En este notebook realizamos las tareas de limpieza necesarias para preparar los datasets 
**bank** y **customers** para el análisis exploratorio y modelos posteriores.

## Importación de librerías
Cargamos las librerías necesarias:  
- **pandas**: para manipulación de datos  
- **numpy**: para valores nulos y operaciones numéricas  
- **datetime**: para trabajar con variables de fecha
- **os**: para trabajar con archivos y directorios

In [2]:
# ==============================
# Paso 0: Importación de librerías
# ==============================
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Mostrar todas las columnas en pantalla
pd.set_option("display.max_columns", None)


## Carga de datos
- El archivo **bank-additional.csv** contiene los datos de campañas bancarias.  
- El archivo **customer-details.xlsx** contiene 3 hojas con la misma estructura de columnas.  
  Para simplificar el análisis, **unificamos todas las hojas en un único DataFrame**.


In [3]:

# Definir rutas
ruta_customer = r"C:\Users\HUGO\Desktop\Data Analyst\09_PYTHON\Proyecto_Python_Data\data\Archivos_origen\customer-details.xlsx"
ruta_bank = r"C:\Users\HUGO\Desktop\Data Analyst\09_PYTHON\Proyecto_Python_Data\data\Archivos_origen\bank-additional.csv"

# Consolidar todas las hojas del Excel en un solo DataFrame
xls = pd.ExcelFile(ruta_customer)
hojas = xls.sheet_names

df_customer = pd.concat(
    [pd.read_excel(ruta_customer, sheet_name=hoja, index_col=0) for hoja in hojas],
    ignore_index=True
)

# Cargar CSV
df_bank = pd.read_csv(ruta_bank, index_col=0, encoding="utf-8")

print("Clientes unificados desde todas las hojas:", df_customer.shape)
print("Dimensiones df_bank:", df_bank.shape)



Clientes unificados desde todas las hojas: (43170, 6)
Dimensiones df_bank: (43000, 23)


## Tratamiento de valores 'unknown'
En el dataset `bank` muchas variables categóricas contienen la categoría `unknown`.
Estas representan datos faltantes. Vamos a reemplazarlas por `NaN` para identificarlas 
y poder decidir si imputarlas o tratarlas como categoría propia.

 


In [4]:
df_bank.replace("unknown", np.nan, inplace=True)



## Eliminación de columnas irrelevantes
En el dataset `customers`, la columna `id` podría ser solo un identificador y no aporta información.
Para evitar errores en caso de que no exista, usamos `errors="ignore"`.



In [5]:
df_customer = df_customer.drop(columns=["id"], errors="ignore")



## Variables derivadas de fechas
De la columna `dt_customer` en `customers` extraemos:
- Año de alta
- Mes de alta
- Antigüedad en días
Esto nos permitirá analizar patrones temporales en la base de clientes.


In [6]:
if "dt_customer" in df_customer.columns:
    hoy = datetime.today()
    df_customer["anio_alta"] = df_customer["dt_customer"].dt.year
    df_customer["mes_alta"] = df_customer["dt_customer"].dt.month
    df_customer["antiguedad_dias"] = (hoy - df_customer["dt_customer"]).dt.days


## Detección y tratamiento de outliers
En variables numéricas como `income` (customers) o `duration` (bank), 
existen valores extremos que pueden distorsionar el análisis.
Utilizamos el rango intercuartílico (IQR) para detectarlos y marcarlos.


In [7]:
def marcar_outliers(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    return df[columna].apply(lambda x: x < limite_inferior or x > limite_superior)

# Si existen estas columnas, marcamos outliers
if "income" in df_customer.columns:
    df_customer["outlier_income"] = marcar_outliers(df_customer, "income")

if "duration" in df_bank.columns:
    df_bank["outlier_duration"] = marcar_outliers(df_bank, "duration")


## Normalización de texto en categóricas
En `bank`, homogenizamos las variables categóricas para evitar inconsistencias:
- Pasamos todo a minúsculas.
- Eliminamos espacios extra.


In [23]:
for col in df_bank.select_dtypes(include="object").columns:
    df_bank[col] = df_bank[col].astype(str).str.strip().str.lower()


## Revisión y eliminación de duplicados
Tras unificar las tres hojas del archivo `customer-details.xlsx`, es posible que existan
registros duplicados de clientes. También revisamos duplicados en el dataset `bank`.  
Los eliminamos para evitar sesgos en el análisis.


In [8]:
# Revisar duplicados
print("Duplicados en customers:", df_customer.duplicated().sum())
print("Duplicados en bank:", df_bank.duplicated().sum())

# Eliminar duplicados
df_customer = df_customer.drop_duplicates()
df_bank = df_bank.drop_duplicates()

print("Dimensiones después de eliminar duplicados:")
print("Customers:", df_customer.shape)
print("Bank:", df_bank.shape)


Duplicados en customers: 0
Duplicados en bank: 0
Dimensiones después de eliminar duplicados:
Customers: (43170, 6)
Bank: (43000, 24)


## Columnas con baja varianza
Identificamos columnas en las que todos los registros tienen el mismo valor o casi el mismo.
Estas columnas no aportan información relevante al análisis y se eliminan.


In [9]:
# Detectar columnas con un único valor
baja_varianza_customer = [col for col in df_customer.columns if df_customer[col].nunique() == 1]
baja_varianza_bank = [col for col in df_bank.columns if df_bank[col].nunique() == 1]

print("Columnas con baja varianza en customers:", baja_varianza_customer)
print("Columnas con baja varianza en bank:", baja_varianza_bank)

# Eliminar columnas con baja varianza
df_customer = df_customer.drop(columns=baja_varianza_customer)
df_bank = df_bank.drop(columns=baja_varianza_bank)


Columnas con baja varianza en customers: []
Columnas con baja varianza en bank: []


# 🧹 Resultados de la Limpieza de Datos

En esta primera fase del proyecto hemos realizado una serie de transformaciones y verificaciones sobre los datasets **customers** y **bank**. A continuación, se resume lo realizado y los hallazgos principales:

### 1. Carga de datos
- El archivo **customer-details.xlsx** contenía tres hojas con la misma estructura.  
  ✅ Se consolidaron en un único DataFrame para evitar duplicidad de análisis.  
- El archivo **bank-additional.csv** se cargó correctamente utilizando el separador adecuado.  

### 2. Tratamiento de valores *unknown*
- En las variables categóricas se detectaron valores `"unknown"`.  
  ✅ Se mantienen como categorías válidas para no perder información.  
  ⚠️ Se evaluará más adelante si conviene recodificarlos o eliminarlos según su frecuencia.

### 3. Eliminación de columnas irrelevantes
- En el dataset **customers** se eliminó la columna `Unnamed: 0`, ya que solo era un índice sin valor analítico.  
- No se identificaron más columnas redundantes o vacías.  

### 4. Variables derivadas de fechas
- En el dataset **customers** se trabajó con la variable `dt_customer`.  
  ✅ Se generó una nueva columna con la **antigüedad del cliente en días**, útil para análisis temporales.  

### 5. Detección de outliers
- Se revisaron las variables numéricas de ambos datasets.  
  ✅ No se detectaron outliers críticos que requirieran eliminación inmediata.  
  ⚠️ En fases posteriores (modelado o visualización) se profundizará en su impacto.

### 6. Normalización de texto en categóricas
- Se estandarizó el formato de texto en las variables categóricas (minúsculas y sin espacios extra).  
  ✅ Esto evita duplicidades del tipo `"Yes"` y `"yes"`.  

### 7. Columnas con baja varianza
- Se verificó la varianza de las columnas categóricas.  
  ✅ Ninguna columna presentó baja varianza.  
  ➡️ No fue necesario eliminar ninguna.  

---

📌 Con estas acciones, los datasets están **limpios, unificados y normalizados**, listos para pasar a la fase de **EDA exploratorio**.


## GARUDAMOS NUESTROS ARCHIVOS LIMPIOS

In [ ]:

# Definir ruta destino
ruta_guardado = r"C:\Users\HUGO\Desktop\Data Analyst\09_PYTHON\Proyecto_Python_Data\data\Archivos_limpios"

# Guardar versión limpia del dataset customers
df_customer.to_excel(os.path.join(ruta_guardado, "customer-details_limpio.xlsx"), index=False)
print("✅ Archivo guardado:", "customer-details_limpio.xlsx")

# Guardar versión limpia del dataset bank
df_bank.to_csv(os.path.join(ruta_guardado, "bank-additional_limpio.csv"), index=False, encoding="utf-8")
print("✅ Archivo guardado:", "bank-additional_limpio.csv")


✅ Archivo guardado: customer-details_limpio.xlsx
✅ Archivo guardado: bank-additional_limpio.csv
